In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

#Load the image
image = cv.imread("the_berry_farms_sunflower_field.jpeg")
# try to see why we need to reduse the intensity

gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

# Define Laplacian parameters
sigma = 3
half_width = 3 * sigma
X,Y = np.meshgrid(np.arange(-half_width, half_width+1), np.arange(-half_width, half_width+1))

# Create the Laplacian filter
Laplacian_filter = 1/(2*np.pi*sigma**2) * (X**2 + Y**2 - 2*sigma**2) * np.exp(-(X**2 + Y**2)/(2*sigma**2))

 